## Useful links:

Rest Browser: https://system.netsuite.com/help/helpcenter/en_US/APIs/REST_API_Browser/record/v1/2023.1/index.html

In [ ]:
#parameters
import oauthlib.oauth1
from requests_oauthlib import OAuth1
import requests
import json
import pyautogui
import time
import webbrowser
import os

In [ ]:
# creds... out in the open... splendid....
consumer_key = ''
consumer_sec = ''
token_key = ''
token_sec = ''
signature_meth = ''
realm = ''

In [ ]:
# Authentication
auth = OAuth1(consumer_key, consumer_sec, token_key, token_sec, signature_type = 'auth_header',
             realm =realm, signature_method = signature_meth)

In [ ]:
base_uri = 'https://.suitetalk.api.netsuite.com'
mid_uri = '/services/rest/record/v1'
endpoint = '/invoice'

In [ ]:
uri1 = 'https://.suitetalk.api.netsuite.com/services/rest/record/v1/invoice?limit=1000&offset=1000'

In [ ]:
base_uri + mid_uri + endpoint

In [ ]:
response = requests.get(base_uri + mid_uri + endpoint, auth = auth).json()

In [ ]:
invoice_list = []
for i in response['links']:
    sub_response = requests.get(i['href'], auth = auth).json()
    for a in sub_response['items']:
        invoice_list.append(a['id'])
        

In [ ]:
url = 'https://.app.netsuite.com/app/accounting/print/hotprint.nl?regular=T&sethotprinter=T&formnumber=165&whence=&printtype=transaction&trantype=custinvc&recordtype=&label=Invoice&id='

In [ ]:
save_path = 'C:/Users/Finance/test/'
chrome_path = 'chrome.exe %s'
for i in invoice_list:
    webbrowser.get(chrome_path).open(url + i)
    time.sleep(5)
    pyautogui.hotkey('ctrl', 's')
    time.sleep(1)
    pyautogui.typewrite(i + '.pdf')
    time.sleep(1)
    pyautogui.hotkey('enter')
    time.sleep(1)
    pyautogui.hotkey('ctrl', 'w')


## Comparison of what was downloaded and whats available

In [ ]:
# full list of invoices in NS
len(invoice_list)

In [ ]:
file_path = '~/IS_NS_Backup/Invoices'
file_list = os.listdir(file_path)

In [ ]:
downloaded_list = []
for i in file_list:
    downloaded_list.append(i.split('.')[0])
    

In [ ]:
# what was dowloaded
len(downloaded_list)

In [ ]:
residual = [i for i in invoice_list if i not in downloaded_list]

In [ ]:
# invoiceIDs that were not downloaded
residual

### Getting Inoice Ledger

In [ ]:
invoice_url = 'https://.suitetalk.api.netsuite.com/services/rest/record/v1/invoice/'

In [ ]:
invoice_ledger = []
for i in invoice_list:
    sublist = []
    invoice = requests.get(invoice_url + i, auth = auth).json()
    sublist.append(i)
    sublist.append(invoice['subsidiary']['refName'])
    sublist.append(invoice['tranId'])
    sublist.append(invoice['entity']['refName'])
    sublist.append(invoice['createdDate'].split('T')[0])
    invoice_ledger.append(sublist)